In [1]:
import torch
from torch import nn

In [3]:
t1 = torch.randn([64,2048,8,4])

In [3]:
t1 = t1.permute(0,2,3,1)

In [4]:
t1.shape

torch.Size([64, 8, 4, 2048])

In [5]:
m = nn.AdaptiveAvgPool2d((4,256))

In [6]:
t1 = m(t1)

In [7]:
t1.shape

torch.Size([64, 8, 4, 256])

In [8]:
t1 = t1.permute(0,3,1,2)

In [9]:
t1.shape

torch.Size([64, 256, 8, 4])

In [86]:
gt = torch.randn(48,2048,8,4)

In [87]:
gt = gt.permute(0,3,1,2)

In [88]:
gt.shape

torch.Size([48, 4, 2048, 8])

In [89]:
m = nn.AdaptiveAvgPool2d((256,8))

In [90]:
gt = m(gt)

In [91]:
gt.shape

torch.Size([48, 4, 256, 8])

In [92]:
gt = gt.permute(0,2,3,1)

In [93]:
gt.shape

torch.Size([48, 256, 8, 4])

In [11]:
t2 = torch.randn([64,2048,8,4])

In [85]:
t2 = t2.permute(0,2,3,1)

NameError: name 't2' is not defined

In [13]:
t2.shape

torch.Size([64, 8, 4, 2048])

In [14]:
t2 = m(t2)

In [15]:
t2.shape

torch.Size([64, 8, 4, 256])

In [16]:
t2 = t2.permute(0,2,3,1)

In [17]:
t2.shape

torch.Size([64, 4, 256, 8])

In [1]:
def Flatten(input):
        return input.contiguous().view(input.size(0), -1)

In [22]:
g = Flatten(t1)

In [23]:
g.shape

torch.Size([64, 8192])

In [24]:
y = Flatten(t2)

In [25]:
y.shape

torch.Size([64, 8192])

In [26]:
u = g-y

In [27]:
v = torch.unsqueeze(u,2)

In [28]:
v.shape

torch.Size([64, 8192, 1])

In [31]:
w = v.permute(0,2,1)

In [32]:
w.shape

torch.Size([64, 1, 8192])

In [2]:
from __future__ import print_function,absolute_import

import numpy as np
import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn 
from torch.utils.data import DataLoader
from torch.optim import lr_scheduler
from torch.nn import functional as F

import os
import sys
import time
import datetime
import argparse
import os.path as osp

import dataset_manager
from dataset_loader import ImageDataset
import transforms as tfms 
from models import hybrid_convnet2,ResNet
from clr import CyclicLR
from loss import CrossEntropy,TripletLoss,DeepSuperVision
from utils import AverageMeter,Logger,save_checkpoint
from metrics import evaluate
from samplers import RandomIdentitySampler

/home/prajjwalb/anaconda3/envs/pytorch/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
dataset_name = 'market1501'
split_id=0
cuhk03_labeled = True
cuhk03_classic_split = False
height = 256
width = 128
workers=4
split_id=0
max_epoch=60
train_batch = 32
test_batch = 32
lr = 0.0003
stepsize= 60
gamma = 0.1
margin=0.3
weight_decay=5e-4
print_freq = 10
num_instances=4
eval_step=20
start_eval=0
start_epoch=0
split_id=0
PATH = 'log'
pin_memory = True

In [4]:
dataset = dataset_manager.init_img_dataset(
    root='data',name=dataset_name,split_id=split_id,
    cuhk03_labeled=cuhk03_labeled,cuhk03_classic_split=cuhk03_classic_split,
)

=> Market1501 loaded
Dataset statistics:
  ------------------------------
  subset   | # ids | # images
  ------------------------------
  train    |   751 |    12936
  query    |   750 |     3368
  gallery  |   751 |    15913
  ------------------------------
  total    |  1501 |    32217
  ------------------------------


In [5]:
tfms_train = tfms.Compose([
    tfms.Random2DTranslation(height,width),
    tfms.RandomHorizontalFlip(),
    tfms.ToTensor(),
    tfms.Normalize(mean=[0.485,0.456,0.406],std=[0.229,0.224,0.225]),
])

tfms_test = tfms.Compose([
    tfms.Resize(size=(height,width)),
    tfms.ToTensor(),
    tfms.Normalize(mean=[0.485,0.456,0.406],std=[0.229,0.224,0.225]),
])

In [6]:
trainloader = DataLoader(
    ImageDataset(dataset.train,transform=tfms_train),
    sampler = RandomIdentitySampler(dataset.train,num_instances=num_instances),
    batch_size = train_batch,num_workers=workers,
    pin_memory=True,drop_last=True,
)

queryloader = DataLoader(
    ImageDataset(dataset.query,transform=tfms_test),
    batch_size=test_batch,shuffle=False,num_workers=workers,
    pin_memory=pin_memory,drop_last=False,
)

galleryloader = DataLoader(
    ImageDataset(dataset.gallery,transform=tfms_test),
    batch_size=test_batch,shuffle=False,num_workers=workers,
    pin_memory=pin_memory,drop_last=False, 
)

In [7]:
t = torch.randn([64,256,8,4])

In [7]:
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

In [8]:
activation = {}
def get_activation(name):
    def hook(model, input, output):
        activation[name] = output.detach()
    return hook

In [9]:
model = hybrid_convnet2.hybrid_cnn().to(device)
model.fc0.conv2.register_forward_hook(get_activation('fc0.conv2'))

In [23]:
person_per_batch = 8
imgs_per_person = 4


bmask = []
l_all_pos = []
l_all_neg = []
pos_targets = torch.Tensor()
neg_targets = torch.Tensor()
C_pos = torch.zeros([train_batch,256,2,4],device=device)
C_neg = torch.zeros([train_batch,256,2,4],device=device)

for batch, (imgs,pids,camids) in enumerate(trainloader):
    
    pids = torch.Tensor.numpy(pids)
    #print(pids)
    camids = torch.Tensor.numpy(camids)
    uid = list(set(pids))
    #print(len(uid))
    #print(uid)
    mask = np.zeros([2*person_per_batch,person_per_batch*imgs_per_person])
    for i in range(len(uid)):
        sel = uid[i]
        # print(sel)
        pos = -1
        neg = -1
        k = -1
        for j in range(len(pids)):
            if (pids[j]==sel):
                k = j
                break
                
        for j in range(len(pids)):
            if (pids[k]==pids[j] and camids[k]!= camids[j]):   # Same IDs and diff cam IDs
                pos = j                        #Postive        
                break
                
        for j in range(len(pids)):
            if (pids[k]!=pids[j]):             #Negative                # Diff Cam IDs
                neg = j
                break

        mask[2*i][k] = 1
        mask[2*i][pos] = 1
        mask[2*i+1][k] = 1
        mask[2*i+1][neg] = 1

    bmask.append(mask)
    #bmask = np.asarray(bmask)
    #print(bmask.shape)
    #bmask = np.asarray(bmask)
    
    l_batch_pos = []
    l_batch_neg = []
    kl = mask  #bmask[batch]
    for i in range(len(kl)):
        l5 = []
        for j in range(len(kl[i])):
            if (kl[i][j]==1):
                l5.append(j)
        if i %2 <1:
            l_batch_pos.append(l5)
        else:
            l_batch_neg.append(l5)
    l_all_pos.append(l_batch_pos)
    l_all_neg.append(l_batch_neg)
    #print(l_all_pos)
    #print(mask[-1])
    #break
    op = model(imgs.cuda())
    f = activation['fc0.conv2']
    f = f.permute(0,3,1,2)
    m = nn.AdaptiveAvgPool2d((256,2))
    f = m(f)
    f = f.permute(0,2,3,1)
    
    for i in range(len(l_batch_pos)):
        pos_idx0 = l_batch_pos[i][0]
        pos_idx1 = l_batch_pos[i][1]
        #print(f[pos_idx0].shape)
        pos_targets = torch.sub(f[pos_idx1],f[pos_idx0])
        C_pos += pos_targets
        #print(pos_targets.shape)
    #pos_targets = torch.Tensor(pos_targets)
    
    for i in range(len(l_batch_neg)):
        neg_idx0 = l_batch_neg[i][0]
        neg_idx1 = l_batch_neg[i][1]
        neg_targets = torch.sub(f[neg_idx1],f[neg_idx0])
        C_neg += neg_targets
    #neg_targets = torch.FloatTensor(neg_targets)
    #print(C_pos.shape)
    g = Flatten(C_pos)
    #print(g.shape)
    y = Flatten(C_neg)
    u = g-y
    v = torch.unsqueeze(u,2)
    w = v.permute(0,2,1)
    x_net = torch.matmul(v,w)
    
    #l_all_pos = np.asarray(l_all_pos)
    #l_all_neg = np.asarray(l_all_neg)
    y  = torch.sum(x_net)
    y = F.relu(y)
    alpha = 1e-9
print(alpha*y)
print(x_net.shape)

            

tensor(0.2998, device='cuda:0')
torch.Size([32, 2048, 2048])


In [22]:
# print(sys.getsizeof(f))
len(bmask[0][])

64

In [49]:
l_all_pos.shape

(46, 16, 2)

In [10]:
m0 = bmask[0]

In [11]:
bmask.shape

(46, 32, 64)

In [12]:
m0[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [13]:
m0[1]

array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [14]:
m0[2]

array([1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [15]:
m0[3]

array([1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [16]:
m0[4]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [17]:
m0[5]

array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [18]:
m0.shape

(32, 64)

In [19]:
l = bmask[0]
print(l.shape)

(32, 64)


In [20]:
t5 = torch.randn([64,256,8,4])

In [17]:

l_all_pos = []
l_all_neg = []

for batch, (images,pids,camids) in enumerate(trainloader):
    l_batch_pos = []
    l_batch_neg = []
    kl = bmask[batch]
    for i in range(len(kl)):
        l5 = []
        for j in range(len(kl[i])):
            if (kl[i][j]==1):
                l5.append(j)
        #print(l5)
        if i %4 <2:
            l_batch_pos.append(l5)
        else:
            l_batch_neg.append(l5)
    l_all_pos.append(l_batch_pos)
    l_all_neg.append(l_batch_neg)

l_all_pos = np.asarray(l_all_pos)
l_all_neg = np.asarray(l_all_neg)

IndexError: list index out of range

In [45]:
l_all_pos.shape

(46, 16, 2)

In [ ]:
for batch, (images,pids,camids) in enumerate(trainloader):
    for i in bmask[batch]:
        

In [ ]:
l_all

In [ ]:
l_pos = []
l_neg = []
pos=0
neg=0
for i in range(len(l_all)):
    if i %4 <2:
        l_pos.append(l_all[i])
    else:
        l_neg.append(l_all[i])

In [ ]:
t.shape

In [ ]:
gh = [t[l_pos[i]] for i in range(len(t))]

In [103]:
gh

[tensor([[[ 1.5173,  1.0364,  0.7829, -0.9682],
          [-0.7907,  0.7564, -0.1425, -0.8597],
          [ 1.6933, -0.1472,  0.5620,  0.0381],
          ...,
          [-0.9840, -0.6046,  0.2014, -0.7626],
          [ 0.0270, -0.4530,  2.2852,  0.3474],
          [-0.5475, -0.5511, -0.7866, -0.5773]],
 
         [[-1.1408,  1.7177,  0.0596,  1.0414],
          [ 0.0229,  0.0119,  2.2627,  0.5056],
          [ 1.4307,  0.0108,  1.1598,  1.4907],
          ...,
          [-0.1542,  0.7712,  0.5923,  0.3264],
          [ 0.9084,  0.3938,  0.9208,  0.3109],
          [ 0.4667,  1.2327,  1.2252, -1.4153]],
 
         [[-0.0196,  0.8295, -1.4581,  0.1806],
          [ 0.2731,  0.9364, -0.2764, -2.1546],
          [-1.5358, -0.0397,  0.8032, -1.6096],
          ...,
          [-0.2238,  0.6670, -0.3275, -1.6469],
          [-0.4121, -0.5381,  0.2720, -0.7533],
          [-0.2202,  0.6838, -0.2347, -0.1572]],
 
         ...,
 
         [[-1.4728,  0.2849, -0.0761,  1.3284],
          [ 0.2257

In [104]:
gn = [(t[l_neg[i]]).shape for i in range(len(t))]

In [105]:
len(gn)

64